In [4]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D,Input
from keras.optimizers import SGD,Adam
from skimage.measure import compare_ssim as ssim

Using TensorFlow backend.


In [5]:
# Define function to determine peak signal-to-noise ratio

def pnsr(target,ref):
    
    #Convert Data Type of Images
    target_data = target.astype("float")
    ref_data = ref.astype("float")
    
    diff = ref_data - target_data
    diff=diff.flatten('C')
    print(diff)
    rmse = math.sqrt(np.mean(diff **2.))
    
    return 20* math.log10(255. /rmse)

In [6]:
# Define function to determine mean squared error

def mse(target,ref):
    target_data = target.astype("float")
    ref_data = ref.astype("float")
    
    err= np.sum((target_data -ref_data)**2)
    err /= float(target.shape[0]*target.shape[1])
    
    return err
    

In [7]:
# Function to compare all 3 image quality metrics
def compare_images(target,ref):
    scores=[]
    scores.append(pnsr(target,ref))
    scores.append(mse(target,ref))
    scores.append(ssim(target,ref,multichannel=True))
    
    return scores

In [11]:
# Prepare Images
def prepare_images(path,factor):
    
    #loop through files
    for file in os.listdir(path):
        # Open the file
        
        img=cv2.imread(path+'/'+file)
    
    #find image dimensions
        h,w,c = img.shape
        nh= h/factor
        nw= w/factor
    
    # resize image down
        img=cv2.resize(img,(nh,nw),interpolation=cv2.INTER_LINEAR)
    
     # resize image UP
        img=cv2.resize(img,(h,w),interpolation=cv2.INTER_LINEAR)
    
    #save the file
    
        print("Saving {}".format(file))
        cv2.imwrite('images/{}'.format(file),img)
    
    

In [12]:
prepare_images('source/',2)

TypeError: integer argument expected, got float